<a href="https://colab.research.google.com/github/MoriyaBitton/Machine_learning/blob/main/Final_Project_Machin_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **<u>Final Project - Machin Learning</u>**

---



---



> **Moriya Bitton** &emsp; **ID: 316451749** 
<br>**Hay Hoffman** &emsp; **ID: 203265186**

## **<u>Introduction</u>**

Machine learning (ML) algorithms are a group of algorithms with a common feature of linearity, which perform regression and classification tasks, usually for high-dimensional data, by finding the rule they represent. 
<br>In this algorithm, a line or hyperplane is used to separate the data into different labels (for classification) or to construct an equation that predicts the future values of a given point in the data (for regression), based on other points' values. Unlike high-dimensional data, images have a grid structure and are defined by their connections between pixels, which makes conventional ML algorithms unsuitable for image labeling. 

Deep neural networks (DNN) are nonlinear algorithms that use convolution neural networks (CNN) to overcome ML's obstacles. In CNN's, pixels within an image are learned spatially by multiplying learnable matrices along the image called kernels, reducing their size, and activating nonlinear functions. 

The extraction of features from data is an important step in any learning process, whether it is deep learning (DL) or machine learning (ML). In most algorithms, it is part of their operation, but for others, it is their primary function. Autoencoders and principal component analysis (PCA) are two examples of these algorithms.

By reducing the dataset dimension, PCA improves interpretability and minimizes information loss. An eigenvalue/eigenvector problem is solved by PCA, resulting in the creation of new variables that maximize dataset variance.

An autoencoder reduces the dimensionality of data by focusing on relevant areas. In this process, input data is compressed, encoded, and then reconstructed as an output to reduce noise. In an autoencoder, incoming data is systematically reduced in complexity by multiple layers inside a neural network, resulting in a dramatically compressed version of the original. The decoding stage reconstructs the encoded data, with the output representing the input equivalently.  

---



---

## **<u>Project's objectives:</u>**

We aim to answer two questions:

1. Are traditional machine learning algorithms capable of classifying images despite their limitations?

2. How do feature extraction algorithms perform better as preprocessors?

<br>In our first experiment, we will only reshape and normalize images. <br>Next, we will feed the extracted data into the algorithm to compress it. 

<br>To test our research question, we will use the most common machine learning algorithms and compare their results with a simple deep learning network.
<br>We will train and test using the handwritten Mnisnt digits, which contain black and white 28x28 images with more than 70,000 images.
<br>Our DL models are built using TensorFlow and open-source libraries like sklearn.

---



---



#### **Impotrs**


In [1]:
# Basic 
import keras
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# Dataset 
from keras.datasets import mnist

# Model 
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier  

from keras.utils import np_utils
from tensorflow.python.keras import layers
from tensorflow.python.keras.models import Model

from keras.models import Sequential
from keras.layers import Dense, Dropout, Input

from sklearn import metrics
from sklearn.metrics import accuracy_score

#### **MNIST - DataSet**

[Wikipedia page](https://en.wikipedia.org/wiki/MNIST_database)


In [2]:
# loading

(x_train, y_train), (x_test, y_test) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


<u>Data reshaping and normalization</u>




In [3]:
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

In [4]:
# shape of dataset

print('X_train: ' + str(x_train.shape), '\tY_train: ' + str(y_train.shape))
print('X_test:  ' + str(x_test.shape), '\tY_test:  ' + str(y_test.shape))

X_train: (60000, 784) 	Y_train: (60000,)
X_test:  (10000, 784) 	Y_test:  (10000,)


In [5]:
def plotting(history):
    colors = {'loss':'r', 'accuracy':'b', 'val_loss':'m', 'val_accuracy':'g'}
    plt.figure(figsize=(10,6))
    plt.xlabel("Epoch")

    for measure in history.keys():
        color = colors[measure]
        ln = len(history[measure])
        plt.plot(range(1,ln+1), history[measure], color + '-', label=measure)  # use last 2 values to draw line

    plt.legend(loc='upper left', scatterpoints = 1, frameon=False)



---



---



## **<u>Experiment 1: DL model vs. classic ML model without data preprocessing </u>**
<br>

  

*   Neural networks<br>
    We present a simple, fully-connected neural network constructed from a densely-connected NN layer with a relu activation function, followed by a dropout layer and a softmax layer at the output. We use the cross-entropy loss, which is the most common loss for classification problems, with adam  as an optimizer. We train the model with 128 batch sizes and 100 epochs.
    <br>
    
*   Logistic Regression <br>
    In statistical analysis, logistic regression predicts whether a data set    belongs to a predefined class based on prior observations.
    By using the sigmoid function, logistic regression can set model output from 0 to 1, and the logistic loss is applied to penalize data classification errors.
    <br>

*   KNN <br>
    A k-nearest neighbor algorithm solves classification and regression problems by assuming that similar things exist nearby.
    Using a distance function, the algorithm classifies a new sample with the label of the k nearest neighbors whose distance is the smallest from the dataset.
    <br>

*   SVM <br>
    The Support Vector Machine finds the hyperplane that labels the dataset using a linear model.
    Based on the constant margin from both sides of the hyperplane, the algorithm reduces the plane weights to find the closest point, creating the support vectors.
    Rather than learning the data itself, SVM learns the rules that the data represent instead of the data itself.
    <br>

*   Random Forest <br>
    A random forest is a supervised machine learning algorithm that is constructed from decision tree algorithms. This algorithm is applied in various industries such as banking and e-commerce to predict behavior and outcomes.
    <br>

---



---



In [6]:
y_train = keras.utils.np_utils.to_categorical(y_train, 10)
y_test = keras.utils.np_utils.to_categorical(y_test, 10)

#### **Neural Network Implementation**


In [7]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(784,)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 512)               262656    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 10)                5130      
                                                                 
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________


#### **Models Comparison**

In [ ]:
# DNN - training and evaluation
history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=20,
                    verbose=1,
                    validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)

# Logistic reggression - Training and evaluation 
clf_lr_org = LogisticRegression()
clf_lr_org.fit(x_train, np.argmax(y_train, axis=1))
acc_lr_org = clf_lr_org.score(x_test, np.argmax(y_test, axis=1))

# SVM - Training and evaluation 
clf_svm_org = LinearSVC()
clf_svm_org.fit(x_train, np.argmax(y_train, axis=1))
pred_svm_org = clf_svm_org.predict(x_test)
acc_svm_org = accuracy_score(np.argmax(y_test, axis=1), pred_svm_org)

# SVM with RBF kernel - Training and evaluation 
clf_svm_RBF_org = SVC(kernel='rbf', probability=True, C=1, gamma=0.1)
clf_svm_RBF_org.fit(x_train, np.argmax(y_train, axis=1))
pred_svm_RBF_org = clf_svm_RBF_org.predict(x_test)
acc_svm_RBF_org = accuracy_score(np.argmax(y_test, axis=1), pred_svm_RBF_org) 

# KNN - Training and evaluation 
clf_knn_org = KNeighborsClassifier()
clf_knn_org.fit(x_train, np.argmax(y_train, axis=1))
pred_knn_org = clf_knn_org.predict(x_test)
acc_knn_org = accuracy_score(np.argmax(y_test, axis=1), pred_knn_org)

# Random Forest - Training and evaluation 
clf_rf_org = RandomForestClassifier(n_estimators=120)
clf_rf_org.fit(x_train,np.argmax(y_train, axis=1))
pred_rf_org = clf_rf_org.predict(x_test)
acc_rf_org = accuracy_score(np.argmax(y_test, axis=1), pred_rf_org)

Epoch 1/20
469/469 [==============================] - 14s 28ms/step - loss: 0.2458 - accuracy: 0.9259 - val_loss: 0.1065 - val_accuracy: 0.9654
Epoch 2/20
469/469 [==============================] - 14s 29ms/step - loss: 0.1000 - accuracy: 0.9692 - val_loss: 0.0825 - val_accuracy: 0.9745
Epoch 3/20
469/469 [==============================] - 8s 17ms/step - loss: 0.0734 - accuracy: 0.9767 - val_loss: 0.0726 - val_accuracy: 0.9770
Epoch 4/20
469/469 [==============================] - 8s 16ms/step - loss: 0.0561 - accuracy: 0.9822 - val_loss: 0.0638 - val_accuracy: 0.9818
Epoch 5/20
469/469 [==============================] - 7s 16ms/step - loss: 0.0461 - accuracy: 0.9849 - val_loss: 0.0693 - val_accuracy: 0.9798
Epoch 6/20
469/469 [==============================] - 8s 16ms/step - loss: 0.0406 - accuracy: 0.9869 - val_loss: 0.0652 - val_accuracy: 0.9802
Epoch 7/20
469/469 [==============================] - 7s 16ms/step - loss: 0.0326 - accuracy: 0.9890 - val_loss: 0.0705 - val_accuracy: 0.97

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


#### **Results**

In [ ]:
print("Test results:\n")
print(100*'-')

print("\nDeep neural network accuracy:\t{:.2f}%".format(score[1]*100))
print("\nLogisticl regression accuracy:\t{:.2f}%".format(acc_lr_org*100))
print("\nKNN accuracy:\t{:.2f}%".format(acc_knn_org*100))
print("\nSVM accuracy:\t{:.2f}%".format(acc_svm_org*100))
print("\nSVM with RBF kernel accuracy:\t{:.2f}%".format(acc_svm_RBF_org*100))
print("\nRandom Forest accuracy:\t{:.2f}%".format(acc_rf_org*100))

#### **Conclusions and Observations** 
We see from the results that DNN performs better than classic ML algorithms. We conjectured that the ML algorithms would not be able to accomplish the task of classifying images correctly, but the results show that we were wrong. This result raises another question, for what complexity level of images, such as classifying RGB images, or more extensive datasets such as ImageNet that contains 1000 classes, classic ML algorithms will provide suitable solutions, as observed here. 

Maybe a better research question should have been: "what is the complexity threshold that ML algorithms still manage to result well
?". 

Mabey, the answer to this question will open a window for a better understanding of the relation between DL algorithms and ML Algorithms for image classification. 

Random Forest - Note:

* n_estimators=120: acc: 97.11%
* n_estimators=100: acc: 97.07%
* n_estimators=90: acc: 96.96%
* n_estimators=50: acc: 96.72%
* n_estimators=20: acc: 95.97%

Even after increasing the value for n_estimators, the accuracy stays in the range of 96% - 97%.



---



---



## **<u>Experiment 2: DL model vs. classic ML model with data preprocessing </u>**

#### **Autoencoder Model**

As for the encoder, we present a simple, fully-connected neural network constructed from a densely-connected NN layer with a relu activation function and a softmax layer at the output. We use the cross-entropy loss as well, with adam as an optimizer. We train the model with 128 batch sizes and 100 epochs. 

In [ ]:
encoding_dim = 32  

input_img = tf.keras.Input(shape=(784,))
encoded = tf.keras.layers.Dense(encoding_dim, activation='relu')(input_img)
decoded = tf.keras.layers.Dense(784, activation='sigmoid')(encoded)

autoencoder = tf.keras.Model(input_img, decoded)

# Encoder
encoder = tf.keras.Model(input_img, encoded)

# Decoder
encoded_input = tf.keras.Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = tf.keras.Model(encoded_input, decoder_layer(encoded_input))

# Autoencoder
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
history = autoencoder.fit(x_train, x_train,
                          epochs=20,
                          batch_size=128,
                          shuffle=True,
                          validation_data=(x_test, x_test))

In [ ]:
plotting(history.history)

In [ ]:
encoded_imgs_train = encoder.predict(x_train)
encoded_imgs_test = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs_test)

#### **PCA - Principal Component Analysis**

In [ ]:
pca = PCA(32)
pca.fit(x_train)
pca_train = pca.transform(x_train)
pca_test = pca.transform(x_test)
approximation = pca.inverse_transform(pca_test)

### **<u>PCA vs. Autoencoder Output</u>**

In [ ]:
n = 10  

print("\nOriginal Data:")
plt.figure(figsize=(15, 5))
for i in range(n):
    ax = plt.subplot(3, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

print("\nEncoded Data:")
plt.figure(figsize=(15, 5))
for i in range(n):    
    ax = plt.subplot(3, n, i + 1)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

print("\nPCA Data:")
plt.figure(figsize=(15, 5))
for i in range(n):
    ax = plt.subplot(3, n, i + 1)
    plt.imshow(approximation[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

#### **Neural Network - Encoded Data**
We perform an adaptation for the previously proposed neural network to be compatible with autoencoder output

In [ ]:
encoded_imgs_train_normalized = encoded_imgs_train / np.max(encoded_imgs_train)
print(encoded_imgs_train_normalized[0].shape)
encoded_imgs_test_normalized = encoded_imgs_test / np.max(encoded_imgs_test)

encoded_model = Sequential()
encoded_model.add(Dense(512, activation='relu', input_shape=(encoding_dim,)))
encoded_model.add(Dropout(0.2))
encoded_model.add(Dense(512, activation='relu'))
encoded_model.add(Dropout(0.2))
encoded_model.add(Dense(10, activation='softmax'))

encoded_model.summary()

encoded_model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

#### **Neural Network - PCA Data**

In [ ]:
pca_model = Sequential()
pca_model.add(Dense(512, activation='relu', input_shape=(32,)))
pca_model.add(Dropout(0.2))
pca_model.add(Dense(512, activation='relu'))
pca_model.add(Dropout(0.2))
pca_model.add(Dense(10, activation='softmax'))

pca_model.summary()

pca_model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

#### **Learning Models Comparison**

In [ ]:
# DNN - Training and evaluation

# ENCODED DATA
encoded_history = encoded_model.fit(encoded_imgs_train_normalized, y_train,
                    batch_size=128,
                    epochs=20,
                    verbose=1,
                    validation_data=(encoded_imgs_test_normalized, y_test))

encoded_score = encoded_model.evaluate(encoded_imgs_test_normalized, y_test, verbose=0)

# PCA DATA
PCA_history = pca_model.fit(pca_train, y_train,
                    batch_size=128,
                    epochs=20,
                    verbose=1,
                    validation_data=(pca_test, y_test))
pca_score = pca_model.evaluate(pca_test, y_test, verbose=0)

In [ ]:
# Logistic reggression - Training and evaluation 

# ENCODED DATA
clf_lr_enc = LogisticRegression()
clf_lr_enc.fit(encoded_imgs_train_normalized, np.argmax(y_train, axis=1))
acc_lr_enc = clf_lr_enc.score(encoded_imgs_test_normalized, np.argmax(y_test, axis=1))

# PCA DATA
clf_lr_pca = LogisticRegression()
clf_lr_pca.fit(pca_train, np.argmax(y_train, axis=1))
acc_lr_pca = clf_lr_pca.score(pca_test, np.argmax(y_test, axis=1))

In [ ]:
# KNN - Training and evaluation 

# ENCODED DATA
clf_knn_enc = KNeighborsClassifier()
clf_knn_enc.fit(encoded_imgs_train_normalized, np.argmax(y_train, axis=1))
pred_knn_enc = clf_knn_enc.predict(encoded_imgs_test_normalized)
acc_knn_enc = accuracy_score(np.argmax(y_test, axis=1), pred_knn_enc)

# PCA DATA
clf_knn_pca = KNeighborsClassifier()
clf_knn_pca.fit(pca_train, np.argmax(y_train, axis=1))
pred_knn_pca = clf_knn_pca.predict(pca_test)
acc_knn_pca = accuracy_score(np.argmax(y_test, axis=1), pred_knn_pca)


In [ ]:
# SVM - Training and evaluation 

# ENCODED DATA
clf_svm_enc = LinearSVC()
clf_svm_enc.fit(encoded_imgs_train_normalized, np.argmax(y_train, axis=1))
pred_svm_enc = clf_svm_enc.predict(encoded_imgs_test_normalized)
acc_svm_enc = accuracy_score(np.argmax(y_test, axis=1), pred_svm_enc)

# PCA DATA
clf_svm_pca = LinearSVC()
clf_svm_pca.fit(pca_train, np.argmax(y_train, axis=1))
pred_svm_pca = clf_svm_pca.predict(pca_test)
acc_svm_pca = accuracy_score(np.argmax(y_test, axis=1), pred_svm_pca)

In [ ]:
# SVM with RBF kernel - Training and evaluation 

# ENCODED DATA
clf_svc_enc = SVC(kernel='rbf', probability=True, C=1, gamma=0.1)
clf_svc_enc.fit(encoded_imgs_train_normalized, np.argmax(y_train, axis=1))
pred_svc_enc = clf_svc_enc.predict(encoded_imgs_test_normalized)
acc_svc_enc = accuracy_score(np.argmax(y_test, axis=1), pred_svc_enc)

# PCA DATA
clf_svc_pca = SVC(kernel='rbf', probability=True, C=1, gamma=0.1)
clf_svc_pca.fit(pca_train, np.argmax(y_train, axis=1))
pred_svc_pca = clf_svc_pca.predict(pca_test)
acc_svc_pca = accuracy_score(np.argmax(y_test, axis=1), pred_svc_pca)

In [ ]:
# Random Forest - Training and evaluation 

# ENCODED DATA
clf_rf_enc = RandomForestClassifier(n_estimators=100)
clf_rf_enc.fit(encoded_imgs_train_normalized, np.argmax(y_train, axis=1))
pred_rf_enc = clf_rf_enc.predict(encoded_imgs_test_normalized)
acc_rf_enc = accuracy_score(np.argmax(y_test, axis=1), pred_rf_enc)

# PCA DATA
clf_rf_pca = RandomForestClassifier(n_estimators=100)
clf_rf_pca.fit(pca_train, np.argmax(y_train, axis=1))
pred_rf_pca = clf_rf_pca.predict(pca_test)
acc_rf_pca = accuracy_score(np.argmax(y_test, axis=1), pred_rf_pca)

#### **Results**

In [ ]:
print("\nTest Results\n")
print(100*'-')

print("\nDeep Neural Network Accuracy\n",
      "\n\tEncoded Data:\t{:.2f}%\n".format(encoded_score[1]*100),
      "\n\tPCA Data:\t{:.2f}%\n".format(pca_score[1]*100))
print(100*'-')

print("\nLogisticl Regression Accuracy\n",
      "\n\tEncoded Data:\t{:.2f}%\n".format(acc_lr_enc*100),
      "\n\tPCA Data:\t{:.2f}%\n".format(acc_lr_pca*100))
print(100*'-')

print("\nKNN Accuracy\n",
      "\n\tEncoded Data:\t{:.2f}%\n".format(acc_knn_enc*100),
      "\n\tPCA Data:\t{:.2f}%\n".format(acc_knn_pca*100))
print(100*'-')

print("\nSVM Accuracy\n",
      "\n\tEncoded Data:\t{:.2f}%\n".format(acc_svm_enc*100),
      "\n\tPCA Data:\t{:.2f}%\n".format(acc_svm_pca*100))
print(100*'-')

print("\nSVC Accuracy\n",
      "\n\tEncoded Data:\t{:.2f}%\n".format(acc_svc_enc*100),
      "\n\tPCA Data:\t{:.2f}%\n".format(acc_svc_pca*100))
print(100*'-')

print("\nRandom Forest Accuracy\n",
      "\n\tEncoded Data:\t{:.2f}%\n".format(acc_rf_enc*100),
      "\n\tPCA Data:\t{:.2f}%\n".format(acc_rf_pca*100))
print(100*'-')

#### **Conclusions and Observations** 
In this study, DNN performed better than classic ML algorithms once again. 

The feature extraction, in contrast to intuition, reduced the algorithm's performance, particularly the ML algorithms, which is curious and requires follow-up research that can focus on changing the PCA number of variables and using different autoencoder architectures. 

All algorithms performed better on the PCA's reduced data than on the autoencoder data; this may be because the PCA was limited to only 32 variables that captured the essence of the data, as demonstrated in the comparison part. Another explanation for the results could be that the autoencoder produces more features than the PCA, which leads to either a) overfitting or b) error during training. 

Results indicate that the accuracy of different algorithms is about 1%, which doesn't necessarily mean that one is better than another. 

Future studies could compare the accuracy and PCA number components, as well as different autoencoder architectures.

Random Forest - Note:

* PCA still give a better result then Autoencider.

* We got a better result without data preprocessing

* n_estimators=50: <br>
Encoded Data:	93.93% <br>
PCA Data:	95.09%

* n_estimators=90: <br>
Encoded Data:	94.21% <br>
PCA Data:	95.48%

* n_estimators=100:	<br>
Encoded Data:	94.24% <br>
PCA Data:	95.35%

* n_estimators=120: <br>
Encoded Data:	94.08% <br>
PCA Data:	95.49%